`The following code builds a bidirectional graph and computes the graph coloring.`

In [ ]:
class CSP:
    '''
    Class to solve a problem using CSP approach.
    Contains all the functions relevant to the CSP.
    '''

    def __init__(self, graph):
        self.graph = graph

    def degree_heuristic(self, vertices):
        '''
        orders the vertices of the graph based on the degree 
        and returns the vertex with max degree.
        '''
        return sorted(vertices.items(), key=lambda x: len(x[1]), reverse=True)[0][0]
    
    def minimum_remaining_values_heuristic(self, domains):
        '''
        Orders the vertices for which assignment have to be done
        based on the number of valid values available for them in their domain. 
        '''
        ordering = sorted(domains.items(), key=lambda x: len(x[1]))
        ties = {}
        for index in range(1,len(ordering)):
            if len(ordering[index][1]) == len(ordering[index-1][1]):
                ties[ordering[index][0]] = self.graph[ordering[index][0]]
                ties[ordering[index-1][0]] = self.graph[ordering[index-1][0]]
            else:
                break
        if ties:
            return self.degree_heuristic(ties)
        return ordering[0][0]

    def select_unassigned_variable(self, assignment, domains):
        '''
        Selects the next vertex for assignment from the set of unassigned
        vertices based on mrv heuristic and uses degree heuristic for tie-breaks.
        '''
        unassigned = set(self.graph.keys())-set(assignment.keys())
        unassigned_domains = {ver:domains[ver] for ver in unassigned}
        return self.minimum_remaining_values_heuristic(unassigned_domains)
    
    def least_constraining_value(self, vertex, domains):
        '''
        Orders the domain vlaues of the vertex in a way that it maximizes the
        available domain values for its neighbouring vertices.
        '''
        domain_points = {d_val:0 for d_val in domains[vertex]}
        value_domains = {}
        for adj in self.graph[vertex]:
            value_domains[adj] = {}
            for value in domains[vertex]:
                value_domains[adj][value] = list(set(domains[adj])-set([value]))
            sorted_domain = sorted(value_domains[adj].items(), key=lambda x:len(x[1]), reverse=True)
            if sorted_domain:
                domain_points[sorted(value_domains[adj].items(), key=lambda x:len(x[1]), reverse=True)[0][0]]+=1
        domains[vertex] = list(dict(sorted(domain_points.items(), key=lambda x:x[1], reverse=True)).keys())
        return domains

    def domain_changed(self, adj, values, domains):
        '''
        Check if the domain is changed and if changed return the changed domain.
        '''
        i=0
        revised = False
        while(len(domains[adj])!=0) and i<len(domains[adj]):
            if not (set(values)-set([domains[adj][i]])):
                domains[adj].pop(i)
                revised = True
            else:
                i+=1
        return revised, domains

    def arc_consistency(self, vertex, value, assignment, domains):
        '''
        recursively check the consistency of the neighbouring vertices
        and return false if any of the domains of the adjacent vertices is empty.
        (Same as AC-3 arc consistency method in the textbook)
        '''
        neighbour_unassigned = [(adj, ver)for adj in self.graph[vertex] for ver in (set(self.graph[adj])-set(assignment.keys()))]
        while(len(neighbour_unassigned)!=0):
            ver, adj = neighbour_unassigned.pop(0)
            bool_, domains = self.domain_changed(adj, domains[ver], domains)
            if bool_:
                if not domains[adj]:
                    return False
                neighbour_unassigned+=[(adj, v) for v in (set(self.graph[adj])-set(assignment.keys())-set([ver]))] 
        return True

    def is_valid(self, assignment, vertex, value):
        '''
        Check if this assignment is valid.
        '''
        for adj in self.graph[vertex]:
            if adj in assignment.keys():
                if value == assignment[adj]:
                    return False
        return True

    def reduce_domains(self, vertex, value, domains):
        '''
        Remove the value 'v' from the domains of vertices adjacent to vertex.
        '''
        for adj in self.graph[vertex]:
            if value in domains[adj]:
                domains[adj].remove(value)

    def extend_domains(self, vertex, value, domains):
        '''
        Add the value 'v' to the domains of vertices adjacent to vertex.
        '''
        for adj in self.graph[vertex]:
            if value not in domains[adj]:
                domains[adj].append(value)
    
    def backtracking_search(self, assignment, domains):
        '''
        Main algorithm to implement the above functions.
        1.) Select unassignedd variable/vertex from the avaiable variables.
        2.) order the domains based on least constraing value first
        3.) if the assignment is valis wrt to previous assignments, then move ahead.
        4.) reduce the domains of the adjacent vertices after the assigned value to 
        the current vertex is known to be valid.
        5.) Check arc-consistency and if it returns true recursively call the backtrack algorithm.
        6.) Further, if this assignment does not give a complete solution in the future, \
            repopulate the reduced domains of the adjacent vertices.
        7.) Intermedially, if a solution is found, return the solution.
        '''
        import copy
        if len(assignment.keys()) == len(self.graph.keys()):
            return assignment
        vertex = self.select_unassigned_variable(assignment, domains)
        domains = self.least_constraining_value(vertex, domains)
        for value in domains[vertex]:
            if self.is_valid(assignment, vertex, value):
                assignment[vertex] = value
                self.reduce_domains(vertex, value, domains)
                domains_copy = copy.deepcopy(domains)
                if self.arc_consistency(vertex, value, assignment, domains_copy):
                    result = self.backtracking_search(assignment, domains)
                    if result:
                        return result
                self.extend_domains(vertex, value, domains)
        return False

def make_graph(filename):
    '''
    Construct a graph from the file given as input.
    It constructs a bidirectional graph.
    '''
    with open(filename, encoding='utf-8') as file:
        lines = file.readlines()
        graph = {}
        for line in lines:
            line = line.split()
            if int(line[1]) in graph.keys():
                graph[int(line[1])].append(int(line[2]))
            else:
                graph[int(line[1])] = [int(line[2])]
            if int(line[2]) in graph.keys():
                graph[int(line[2])].append(int(line[1]))
            else:
                graph[int(line[2])] = [int(line[1])]
        for key, value in graph.items():
            graph[key] = list(set(value))
    return graph

def main():
    '''
    Main function
    '''
    filenames = ['anna.col', 'school1.col']
    for filename in filenames:
        print('For file: ', filename)
        graph = make_graph(filename)
        if filename == 'anna.col':
            domains = {ver:[j for j in range(1,12)] for ver in graph.keys()}
        else:
            domains = {ver:[j for j in range(1,43)] for ver in graph.keys()}
        csp_problem = CSP(graph)
        result = csp_problem.backtracking_search(assignment={}, domains=domains)
        if result:
            print(dict(sorted(result.items(), key=lambda x:x[0])))
            print('No: of colors required = ',sorted(result.items(), key=lambda x:x[1])[-1][-1])
        else:
            print("No result")
        print()

if __name__ == '__main__':
    main()


For file:  anna.col
{1: 1, 2: 1, 3: 1, 4: 2, 5: 1, 6: 6, 7: 3, 8: 3, 9: 2, 10: 2, 11: 3, 12: 2, 13: 3, 14: 1, 15: 2, 16: 3, 17: 1, 18: 1, 19: 5, 20: 7, 21: 3, 22: 2, 23: 2, 24: 1, 25: 2, 26: 5, 27: 2, 28: 3, 29: 2, 30: 1, 31: 2, 32: 1, 33: 8, 34: 2, 35: 2, 36: 2, 37: 1, 38: 1, 39: 2, 40: 2, 41: 2, 42: 1, 43: 2, 44: 2, 45: 5, 46: 3, 47: 1, 48: 1, 49: 1, 50: 2, 51: 5, 52: 3, 53: 2, 54: 1, 55: 2, 56: 1, 57: 3, 58: 2, 59: 3, 60: 1, 61: 1, 62: 3, 63: 2, 64: 4, 65: 1, 66: 1, 67: 2, 68: 2, 69: 2, 70: 2, 71: 1, 72: 5, 73: 7, 74: 4, 75: 2, 76: 6, 77: 4, 78: 2, 79: 4, 80: 3, 81: 11, 82: 2, 83: 7, 84: 2, 85: 9, 86: 1, 87: 2, 88: 2, 89: 3, 90: 3, 91: 10, 92: 5, 93: 1, 94: 5, 95: 3, 96: 3, 97: 2, 98: 1, 99: 9, 100: 4, 101: 6, 102: 1, 103: 4, 104: 4, 105: 2, 106: 1, 107: 2, 108: 7, 109: 1, 110: 1, 111: 1, 112: 1, 113: 2, 114: 2, 115: 4, 116: 7, 117: 2, 118: 4, 119: 1, 120: 2, 121: 2, 122: 1, 123: 3, 124: 2, 125: 1, 126: 1, 127: 3, 128: 1, 129: 1, 130: 5, 131: 2, 132: 2, 133: 3, 134: 1, 135: 5, 136: 

`The following code computes the graph coloring by exactly taking the input from the file.`

In [1]:
class CSP:
    '''
    Class to solve a problem using CSP approach.
    Contains all the functions relevant to the CSP.
    '''

    def __init__(self, graph):
        self.graph = graph

    def degree_heuristic(self, vertices):
        '''
        orders the vertices of the graph based on the degree 
        and returns the vertex with max degree.
        '''
        return sorted(vertices.items(), key=lambda x: len(x[1]), reverse=True)[0][0]
    
    def minimum_remaining_values_heuristic(self, domains):
        '''
        Orders the vertices for which assignment have to be done
        based on the number of valid values available for them in their domain. 
        '''
        ordering = sorted(domains.items(), key=lambda x: len(x[1]))
        ties = {}
        for index in range(1,len(ordering)):
            if len(ordering[index][1]) == len(ordering[index-1][1]):
                ties[ordering[index][0]] = self.graph[ordering[index][0]]
                ties[ordering[index-1][0]] = self.graph[ordering[index-1][0]]
            else:
                break
        if ties:
            return self.degree_heuristic(ties)
        return ordering[0][0]

    def select_unassigned_variable(self, assignment, domains):
        '''
        Selects the next vertex for assignment from the set of unassigned
        vertices based on mrv heuristic and uses degree heuristic for tie-breaks.
        '''
        unassigned = set(self.graph.keys())-set(assignment.keys())
        unassigned_domains = {ver:domains[ver] for ver in unassigned}
        return self.minimum_remaining_values_heuristic(unassigned_domains)
    
    def least_constraining_value(self, vertex, domains):
        '''
        Orders the domain vlaues of the vertex in a way that it maximizes the
        available domain values for its neighbouring vertices.
        '''
        domain_points = {d_val:0 for d_val in domains[vertex]}
        value_domains = {}
        for adj in self.graph[vertex]:
            value_domains[adj] = {}
            for value in domains[vertex]:
                value_domains[adj][value] = list(set(domains[adj])-set([value]))
            sorted_domain = sorted(value_domains[adj].items(), key=lambda x:len(x[1]), reverse=True)
            if sorted_domain:
                domain_points[sorted(value_domains[adj].items(), key=lambda x:len(x[1]), reverse=True)[0][0]]+=1
        domains[vertex] = list(dict(sorted(domain_points.items(), key=lambda x:x[1], reverse=True)).keys())
        return domains

    def domain_changed(self, adj, values, domains):
        '''
        Check if the domain is changed and if changed return the changed domain.
        '''
        i=0
        revised = False
        while(len(domains[adj])!=0) and i<len(domains[adj]):
            if not (set(values)-set([domains[adj][i]])):
                domains[adj].pop(i)
                revised = True
            else:
                i+=1
        return revised, domains

    def arc_consistency(self, vertex, value, assignment, domains):
        '''
        recursively check the consistency of the neighbouring vertices
        and return false if any of the domains of the adjacent vertices is empty.
        (Same as AC-3 arc consistency method in the textbook)
        '''
        neighbour_unassigned = [(adj, ver)for adj in self.graph[vertex] for ver in (set(self.graph[adj])-set(assignment.keys()))]
        while(len(neighbour_unassigned)!=0):
            ver, adj = neighbour_unassigned.pop(0)
            bool_, domains = self.domain_changed(adj, domains[ver], domains)
            if bool_:
                if not domains[adj]:
                    return False
                neighbour_unassigned+=[(adj, v) for v in (set(self.graph[adj])-set(assignment.keys())-set([ver]))] 
        return True

    def is_valid(self, assignment, vertex, value):
        '''
        Check if this assignment is valid.
        '''
        for adj in self.graph[vertex]:
            if adj in assignment.keys():
                if value == assignment[adj]:
                    return False
        return True

    def reduce_domains(self, vertex, value, domains):
        '''
        Remove the value 'v' from the domains of vertices adjacent to vertex.
        '''
        for adj in self.graph[vertex]:
            if value in domains[adj]:
                domains[adj].remove(value)

    def extend_domains(self, vertex, value, domains):
        '''
        Add the value 'v' to the domains of vertices adjacent to vertex.
        '''
        for adj in self.graph[vertex]:
            if value not in domains[adj]:
                domains[adj].append(value)
    
    def backtracking_search(self, assignment, domains):
        '''
        Main algorithm to implement the above functions.
        1.) Select unassignedd variable/vertex from the avaiable variables.
        2.) order the domains based on least constraing value first
        3.) if the assignment is valis wrt to previous assignments, then move ahead.
        4.) reduce the domains of the adjacent vertices after the assigned value to 
        the current vertex is known to be valid.
        5.) Check arc-consistency and if it returns true recursively call the backtrack algorithm.
        6.) Further, if this assignment does not give a complete solution in the future, \
            repopulate the reduced domains of the adjacent vertices.
        7.) Intermedially, if a solution is found, return the solution.
        '''
        import copy
        if len(assignment.keys()) == len(self.graph.keys()):
            return assignment
        vertex = self.select_unassigned_variable(assignment, domains)
        domains = self.least_constraining_value(vertex, domains)
        for value in domains[vertex]:
            if self.is_valid(assignment, vertex, value):
                assignment[vertex] = value
                self.reduce_domains(vertex, value, domains)
                domains_copy = copy.deepcopy(domains)
                if self.arc_consistency(vertex, value, assignment, domains_copy):
                    result = self.backtracking_search(assignment, domains)
                    if result:
                        return result
                self.extend_domains(vertex, value, domains)
        return False

def make_graph(filename):
    '''
    Construct a graph from the file given as input.
    It constructs a bidirectional graph.
    '''
    with open(filename, encoding='utf-8') as file:
        lines = file.readlines()
        graph = {}
        for line in lines:
            line = line.split()
            if int(line[1]) in graph.keys():
                graph[int(line[1])].append(int(line[2]))
            else:
                graph[int(line[1])] = [int(line[2])]
            if int(line[2]) not in graph.keys():
                graph[int(line[2])] = []
        for key, value in graph.items():
            graph[key] = list(set(value))
    return graph

def main():
    '''
    Main function
    '''
    filenames = ['anna.col', 'school1.col']
    for filename in filenames:
        print('For file: ', filename)
        graph = make_graph(filename)
        if filename == 'anna.col':
            domains = {ver:[j for j in range(1,12)] for ver in graph.keys()}
        else:
            domains = {ver:[j for j in range(1,43)] for ver in graph.keys()}
        csp_problem = CSP(graph)
        result = csp_problem.backtracking_search(assignment={}, domains=domains)
        if result:
            print(dict(sorted(result.items(), key=lambda x:x[0])))
            print('No: of colors required = ',sorted(result.items(), key=lambda x:x[1])[-1][-1])
        else:
            print("No result")
        print()

if __name__ == '__main__':
    main()


For file:  anna.col
{1: 1, 2: 1, 3: 1, 4: 2, 5: 1, 6: 6, 7: 3, 8: 3, 9: 2, 10: 2, 11: 3, 12: 2, 13: 3, 14: 1, 15: 2, 16: 3, 17: 1, 18: 1, 19: 5, 20: 7, 21: 3, 22: 2, 23: 2, 24: 1, 25: 2, 26: 5, 27: 2, 28: 3, 29: 2, 30: 1, 31: 2, 32: 1, 33: 8, 34: 2, 35: 2, 36: 2, 37: 1, 38: 1, 39: 2, 40: 2, 41: 2, 42: 1, 43: 2, 44: 2, 45: 5, 46: 3, 47: 1, 48: 1, 49: 1, 50: 2, 51: 5, 52: 3, 53: 2, 54: 1, 55: 2, 56: 1, 57: 3, 58: 2, 59: 3, 60: 1, 61: 1, 62: 3, 63: 2, 64: 4, 65: 1, 66: 1, 67: 2, 68: 2, 69: 2, 70: 2, 71: 1, 72: 5, 73: 7, 74: 4, 75: 2, 76: 6, 77: 4, 78: 2, 79: 4, 80: 3, 81: 11, 82: 2, 83: 7, 84: 2, 85: 9, 86: 1, 87: 2, 88: 2, 89: 3, 90: 3, 91: 10, 92: 5, 93: 1, 94: 5, 95: 3, 96: 3, 97: 2, 98: 1, 99: 9, 100: 4, 101: 6, 102: 1, 103: 4, 104: 4, 105: 2, 106: 1, 107: 2, 108: 7, 109: 1, 110: 1, 111: 1, 112: 1, 113: 2, 114: 2, 115: 4, 116: 7, 117: 2, 118: 4, 119: 1, 120: 2, 121: 2, 122: 1, 123: 3, 124: 2, 125: 1, 126: 1, 127: 3, 128: 1, 129: 1, 130: 5, 131: 2, 132: 2, 133: 3, 134: 1, 135: 5, 136: 